# Step 0
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

In [1]:
import pandas as pd
import numpy as np

In [2]:
energy = pd.read_excel('data/Energy Indicators.xls')
# energy.head(20)

In [3]:
# energy.tail(40)

### dropping useless rows

In [4]:
energy.drop(index= list(range(17)) + list(energy.index[-38:]) , columns= energy.columns[:2], inplace=True)
energy

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
17,Afghanistan,321,10,78.6693
18,Albania,102,35,100
19,Algeria,1959,51,0.55101
20,American Samoa,...,...,0.641026
21,Andorra,9,121,88.6957
...,...,...,...,...
239,Viet Nam,2554,28,45.3215
240,Wallis and Futuna Islands,0,26,0
241,Yemen,344,13,0
242,Zambia,400,26,99.7147


### renaming columns

In [5]:
rename_mapper = dict(zip( list(energy.columns) , ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']))
print(rename_mapper)
energy.rename(columns = rename_mapper , inplace = True)

{'Unnamed: 2': 'Country', 'Unnamed: 3': 'Energy Supply', 'Unnamed: 4': 'Energy Supply per Capita', 'Unnamed: 5': '% Renewable'}


In [6]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227 entries, 17 to 243
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Country                   227 non-null    object
 1   Energy Supply             227 non-null    object
 2   Energy Supply per Capita  227 non-null    object
 3   % Renewable               227 non-null    object
dtypes: object(4)
memory usage: 8.9+ KB


### replacing ... with NaN

In [7]:
energy.replace('...',np.NaN, inplace=True)

### Petajoules into Giga joules

In [8]:
energy = energy.astype({'Energy Supply':'float64', 'Energy Supply per Capita':'float64', '% Renewable':'float64'})

In [9]:
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x * 10**6)
energy.head(3)

,Country,Energy Supply,Energy Supply per Capita,% Renewable
17,Afghanistan,3.210000e+08,10.0,78.66928
18,Albania,1.020000e+08,35.0,100.00000
19,Algeria,1.959000e+09,51.0,0.55101


### Cleaning countries' names

In [10]:
def clean_countries(x):
    
    y = ''.join([i for i in x if not i.isdigit()]).strip()
    
    if '(' in x:
        y = x.split('(',1)[0]
        return y.strip()
        
    replacement_dict = dict({"Republic of Korea": "South Korea",
                            "United States of America": "United States",
                            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                            "China, Hong Kong Special Administrative Region": "Hong Kong"})

    if y in replacement_dict.keys():
        return replacement_dict[y]
        
    return y

In [11]:
energy['Country'].apply(clean_countries).str.contains('[\(]|[0-9]').sum()

0

In [12]:
country_list = list(dict({"Republic of Korea": "South Korea",
                            "United States of America": "United States",
                            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                            "China, Hong Kong Special Administrative Region": "Hong Kong"}).keys())
energy['Country'].apply(clean_countries).apply(lambda x: x in country_list).sum()

0

In [13]:
energy.loc[:,['Country']] = energy['Country'].apply(clean_countries)

In [14]:
energy.reset_index(drop=True,inplace=True)

In [15]:
energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


### Step 1
<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

In [16]:
GDP = pd.read_csv('data/world_bank.csv')
GDP.head()

,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Last Updated Date,22/07/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2006.0,2007.0,2008.0,2009.0,2.010000e+03,2011.0,2012.0,2013.0,2014.0,2015.0
4,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN


### Dropping useless rows

In [17]:
GDP.drop(index= list(range(3)), inplace=True)

In [18]:
GDP.columns = list(GDP.iloc[0,:4]) + list(GDP.iloc[0,4:].astype('int64',errors='ignore'))
GDP.columns = GDP.columns.astype(str)
GDP = GDP.iloc[1:]

In [19]:
GDP.reset_index(drop=True,inplace=True)

In [20]:
GDP.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10


### Cleaning countries' names

In [21]:
GDP[GDP['Country Name'].str.contains('Korea|Iran|China',na=False)]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
38,China,CHN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,1.262510e+11,9.178435e+10,8.664442e+10,9.548215e+10,1.129550e+11,1.321580e+11,...,3.992330e+12,4.559040e+12,4.997780e+12,5.459250e+12,6.039660e+12,6.612490e+12,7.124980e+12,7.672450e+12,8.230120e+12,8.798000e+12
93,"Hong Kong SAR, China",HKG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,1.764691e+10,...,2.019160e+11,2.149700e+11,2.195440e+11,2.141450e+11,2.286380e+11,2.396460e+11,2.437210e+11,2.512080e+11,2.577760e+11,2.638610e+11
109,"Iran, Islamic Rep.",IRN,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,5.500820e+10,6.072406e+10,6.552689e+10,7.014134e+10,7.608598e+10,8.904776e+10,...,3.895520e+11,4.250650e+11,4.289910e+11,4.389210e+11,4.677900e+11,4.853310e+11,4.532570e+11,4.445930e+11,4.639030e+11,NaN
123,"Korea, Rep.",KOR,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,2.757696e+10,2.893825e+10,2.964966e+10,3.247578e+10,3.493136e+10,3.674434e+10,...,9.410200e+11,9.924320e+11,1.020510e+12,1.027730e+12,1.094500e+12,1.134800e+12,1.160810e+12,1.194430e+12,1.234340e+12,1.266580e+12
144,"Macao SAR, China",MAC,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.872600e+10,2.143144e+10,2.215878e+10,2.245152e+10,2.812364e+10,3.421877e+10,3.737706e+10,4.156352e+10,4.120936e+10,3.282378e+10
191,"Korea, Dem. People’s Rep.",PRK,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def country_dict_replace(x):
    
    replacement_dict = dict({"Korea, Rep.": "South Korea", 
                                "Iran, Islamic Rep.": "Iran",
                                "Hong Kong SAR, China": "Hong Kong"})

    if x.strip() in replacement_dict.keys():
        return replacement_dict[x.strip()]
    
    return x

In [23]:
GDP.loc[:,['Country Name']] = GDP['Country Name'].apply(country_dict_replace)

In [24]:
GDP.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10


In [25]:
print((GDP['Country Name'] == 'South Korea').sum())
print((GDP['Country Name'] == 'Iran').sum())
print((GDP['Country Name'] == 'Hong Kong').sum())

1
1
1


# Step 2
Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

In [26]:
ScimEn = pd.read_excel('data/scimagojr-3.xlsx')
ScimEn.head(5)

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


# Step 3
Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This step should yeild a DataFrame with 20 columns and 15 entries.*

### Merging and filtering for last 10 years

In [27]:
from functools import reduce

In [28]:
energy.set_index('Country',inplace=True)
GDP.set_index('Country Name',inplace=True)
ScimEn.set_index('Country',inplace=True)

In [29]:
df_list = [ScimEn, energy, GDP.loc[:,['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']] ]

In [30]:
df_joined = reduce(lambda left,right : pd.merge(left, right, left_index=True, right_index=True, how='inner') , df_list)

In [31]:
df_joined

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,3.992330e+12,4.559040e+12,4.997780e+12,5.459250e+12,6.039660e+12,6.612490e+12,7.124980e+12,7.672450e+12,8.230120e+12,8.798000e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,1.479230e+13,1.505540e+13,1.501150e+13,1.459480e+13,1.496440e+13,1.520400e+13,1.554220e+13,1.577370e+13,1.615660e+13,1.654860e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,5.496540e+12,5.617040e+12,5.558530e+12,5.251310e+12,5.498720e+12,5.473740e+12,5.569100e+12,5.644660e+12,5.642880e+12,5.669560e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,2.419630e+12,2.482200e+12,2.470610e+12,2.367050e+12,2.403500e+12,2.450910e+12,2.479810e+12,2.533370e+12,2.605640e+12,2.666330e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,1.385790e+12,1.504070e+12,1.583000e+12,1.459200e+12,1.524920e+12,1.589940e+12,1.645880e+12,1.666930e+12,1.678710e+12,1.616150e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Maldives,183,1,1,1,0,1.00,1,1.500000e+07,43.0,0.00000,1.862196e+09,2.063143e+09,2.320557e+09,2.192661e+09,2.323402e+09,2.525755e+09,2.588810e+09,2.710479e+09,2.886134e+09,2.929826e+09
Belize,185,1,1,6,0,6.00,1,1.300000e+07,39.0,64.69003,1.285312e+09,1.299522e+09,1.341485e+09,1.352206e+09,1.397113e+09,1.426484e+09,1.479846e+09,1.499280e+09,1.560479e+09,1.590417e+09
Palau,186,1,1,0,0,0.00,0,3.000000e+06,152.0,18.75000,2.074094e+08,2.072990e+08,1.957150e+08,1.779528e+08,1.838000e+08,1.930672e+08,1.992454e+08,1.943911e+08,2.026654e+08,2.216412e+08
Guyana,187,1,1,0,0,0.00,0,3.400000e+07,45.0,0.00000,1.919759e+09,2.054533e+09,2.095143e+09,2.164673e+09,2.259288e+09,2.382129e+09,2.496863e+09,2.627240e+09,2.728150e+09,2.810153e+09


### Selecting top 15 countries

In [32]:
df_joined_reduced = df_joined[df_joined['Rank']<16]
df_joined_reduced

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992330e+12,4.559040e+12,4.997780e+12,5.459250e+12,6.039660e+12,6.612490e+12,7.124980e+12,7.672450e+12,8.230120e+12,8.798000e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501150e+13,1.459480e+13,1.496440e+13,1.520400e+13,1.554220e+13,1.577370e+13,1.615660e+13,1.654860e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496540e+12,5.617040e+12,5.558530e+12,5.251310e+12,5.498720e+12,5.473740e+12,5.569100e+12,5.644660e+12,5.642880e+12,5.669560e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419630e+12,2.482200e+12,2.470610e+12,2.367050e+12,2.403500e+12,2.450910e+12,2.479810e+12,2.533370e+12,2.605640e+12,2.666330e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385790e+12,1.504070e+12,1.583000e+12,1.459200e+12,1.524920e+12,1.589940e+12,1.645880e+12,1.666930e+12,1.678710e+12,1.616150e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564470e+12,1.596740e+12,1.612710e+12,1.565140e+12,1.613410e+12,1.664090e+12,1.693130e+12,1.730690e+12,1.773490e+12,1.792610e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332890e+12,3.441560e+12,3.478810e+12,3.283340e+12,3.417300e+12,3.542370e+12,3.556720e+12,3.567320e+12,3.624390e+12,3.685560e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265890e+12,1.374870e+12,1.428360e+12,1.549480e+12,1.708460e+12,1.821870e+12,1.924240e+12,2.051980e+12,2.200620e+12,2.367210e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669420e+12,2.674640e+12,2.595970e+12,2.646990e+12,2.702030e+12,2.706970e+12,2.722570e+12,2.729630e+12,2.761190e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410200e+11,9.924320e+11,1.020510e+12,1.027730e+12,1.094500e+12,1.134800e+12,1.160810e+12,1.194430e+12,1.234340e+12,1.266580e+12


In [33]:
df_joined_reduced.shape

(15, 20)

# Step 4
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This step should yield a single number.*

In [34]:
print('Number of records dropped is {} '.format(len(df_joined) - len(df_joined_reduced)))

Number of records dropped is 147 


# Step 5

#### Answer the following questions in the context of only the top 15 countries by Scimagojr Rank 


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This step should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [35]:
df_joined_reduced.columns[10:]

Index(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object')

In [36]:
avgGDP = df_joined_reduced.loc[:, df_joined_reduced.columns[10:] ].mean(axis=1).sort_values(ascending=False)
avgGDP

United States         1.536435e+13
China                 6.348610e+12
Japan                 5.542208e+12
Germany               3.493026e+12
France                2.681725e+12
United Kingdom        2.487905e+12
Brazil                2.189793e+12
Italy                 2.120175e+12
India                 1.769298e+12
Canada                1.660648e+12
Russian Federation    1.565459e+12
Spain                 1.418079e+12
Australia             1.164041e+12
South Korea           1.106715e+12
Iran                  4.441559e+11
dtype: float64

# Step  6
What is the mean `Energy Supply per Capita`?

*This step should return a single number.*

In [37]:
df_joined_reduced['Energy Supply per Capita'].mean()

157.6

# Step 7
What country has the maximum % Renewable and what is the percentage?

*This step should return a tuple with the name of the country and the percentage.*

In [38]:
tuple(df_joined_reduced['% Renewable'].agg(['idxmax','max']))

('Brazil', 69.64803)

# Step 8
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This step should return a tuple with the name of the country and the ratio.*

In [39]:
df_joined_reduced.loc[:,'Citation Ratio'] = df_joined_reduced['Self-citations']/df_joined_reduced['Citations']
tuple(df_joined_reduced['Citation Ratio'].agg(['idxmax','max']))

C:\Users\oswel\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\oswel\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


('China', 0.6893126179389422)

# Step 9
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This step should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [40]:
df_joined_reduced.loc[:,'HighRenew'] = (df_joined_reduced['% Renewable'] > df_joined_reduced['% Renewable'].median()).astype(int)
HighRenew = df_joined_reduced.sort_values(by='Rank',ascending=True)['HighRenew']

C:\Users\oswel\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\oswel\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [41]:
HighRenew

China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int32

# Step 10
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

### Dataframe with continents

In [42]:
df_continent = df_joined_reduced.copy()
df_continent.index = df_continent.index.set_names(['Country'])

In [43]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [44]:
df_continent['Continent'] = pd.Series(df_continent.index, index=df_continent.index).apply(lambda x:ContinentDict[x.strip()])

In [45]:
df_continent_copy = df_continent

In [46]:
df_continent_copy 

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2009,2010,2011,2012,2013,2014,2015,Citation Ratio,HighRenew,Continent
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,...,5.459250e+12,6.039660e+12,6.612490e+12,7.124980e+12,7.672450e+12,8.230120e+12,8.798000e+12,0.689313,1,Asia
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,...,1.459480e+13,1.496440e+13,1.520400e+13,1.554220e+13,1.577370e+13,1.615660e+13,1.654860e+13,0.335031,0,North America
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,...,5.251310e+12,5.498720e+12,5.473740e+12,5.569100e+12,5.644660e+12,5.642880e+12,5.669560e+12,0.275997,0,Asia
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,...,2.367050e+12,2.403500e+12,2.450910e+12,2.479810e+12,2.533370e+12,2.605640e+12,2.666330e+12,0.183773,0,Europe
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,...,1.459200e+12,1.524920e+12,1.589940e+12,1.645880e+12,1.666930e+12,1.678710e+12,1.616150e+12,0.362517,1,Europe
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,...,1.565140e+12,1.613410e+12,1.664090e+12,1.693130e+12,1.730690e+12,1.773490e+12,1.792610e+12,0.190369,1,North America
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,...,3.283340e+12,3.417300e+12,3.542370e+12,3.556720e+12,3.567320e+12,3.624390e+12,3.685560e+12,0.195111,1,Europe
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,...,1.549480e+12,1.708460e+12,1.821870e+12,1.924240e+12,2.051980e+12,2.200620e+12,2.367210e+12,0.288973,0,Asia
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,...,2.595970e+12,2.646990e+12,2.702030e+12,2.706970e+12,2.722570e+12,2.729630e+12,2.761190e+12,0.218943,0,Europe


### Converting into long format

In [47]:
countries_number = df_continent_copy.groupby('Continent').size()
countries_number = pd.DataFrame(countries_number).rename(columns={0:'size'})

In [48]:
df_continent_copy = df_continent_copy.set_index('Continent').iloc[:,10:-2]

In [49]:
df_continent_copy.sort_index(inplace=True)

In [50]:
df_continent_long = df_continent_copy.reset_index().melt(id_vars='Continent',var_name='years', value_name='population')


In [51]:
df_continent_long

,Continent,years,population
0,Asia,2006,3.992330e+12
1,Asia,2006,5.496540e+12
2,Asia,2006,1.265890e+12
3,Asia,2006,9.410200e+11
4,Asia,2006,3.895520e+11
...,...,...,...
145,Europe,2015,2.049320e+12
146,Europe,2015,1.419820e+12
147,North America,2015,1.654860e+13
148,North America,2015,1.792610e+12


### Performing aggregations

In [52]:
df1 = df_continent_long.groupby('Continent').agg({'population':['sum','mean','std']}).droplevel(0,axis=1)

In [53]:
countries_number.join(df1, on='Continent')

,size,sum,mean,std
Continent,,,,
Asia,5,1.516657e+14,3.095219e+12,2.545449e+12
Australia,1,1.164041e+13,1.164041e+12,9.216578e+10
Europe,6,1.376637e+14,2.294395e+12,7.122466e+11
North America,2,1.702500e+14,8.512499e+12,7.043380e+12
South America,1,2.189793e+13,2.189793e+12,1.993037e+11
